## Primera Etapa
Instalación de las librerías necesarias

In [ ]:
#pip install pymongo
#pip install dnspython
#pip install pandas

## Segunda Etapa
Conexión a la BD Steam MongoDB y colección de documentos Games

In [ ]:
import pymongo
from pandas import DataFrame

client = pymongo.MongoClient(
   "mongodb://mongo1:30001,mongo2:30002,mongo3:30003/?replicaSet=my-replica-set&readPreference=primary&appname=MongoDB%20Compass&ssl=false")

In [ ]:
db = client['Steam']

In [ ]:
collection = db['Games']

#**Consultas**

1 - Juegos lanzados en 2018 (nombre y fecha de lanzamiento, limita a 20 resultados)

In [ ]:
# 1. Juegos lanzados en 2018 (nombre y fecha de lanzamiento, limita a 20 resultados)
def consulta_1():
    print("\nConsulta 1: Juegos lanzados en 2018")
    result = collection.find(
        {"release_date": {"$regex": "2018"}},
        {"_id": 0, "name": 1, "release_date": 1}
    ).limit(20)
    for game in result:
        print(game)

consulta_1()

2 - Juegos con precio entre \$10 y \$50 y al menos un DLC, ordenados por cantidad de DLC en orden descendente (limita a 10 resultados)

In [ ]:
def consulta_2():
    print("\nConsulta 2: Juegos con precio entre $10 y $50 y al menos un DLC")
    result = collection.find(
        {
            "price": {"$gte": 10, "$lte": 50},
            "dlc_count": {"$gt": 0}
        },
        {"_id": 0, "name": 1, "dlc_count": 1}
    ).sort("dlc_count", -1).limit(10)

    for game in result:
        print(game)

consulta_2()

3 - Juegos para mayores de 12 años con calificación de usuario superior a 80 (limita a 15 resultados)

In [ ]:
def consulta_3():
    print("\nConsulta 3: Juegos para mayores de 12 años con calificación > 80")
    result = collection.find(
        {
            "required_age": {"$gt": 12},
            "user_score": {"$gt": 80}
        },
        {"_id": 0, "name": 1, "required_age": 1, "user_score": 1}
    ).limit(15)
    for game in result:
        print(game)

consulta_3()

4 - Los 10 juegos más caros lanzados después de 2015

In [ ]:
def consulta_4():
    print("\nConsulta 4: Los 10 juegos más caros lanzados después de 2015")
    result = collection.find(
        {
            "release_date": {"$regex": "^201[5-9]|202"},
            "price": {"$exists": True}
        },
        {"_id": 0, "name": 1, "price": 1}
    ).sort("price", -1).limit(10)
    for game in result:
        print(game)

consulta_4()

5 - Los 5 juegos con mejor calificación de usuarios para todas las edades (required_age = 0)

In [ ]:
def consulta_5():
    print("\nConsulta 5: Los 5 juegos mejor calificados para todas las edades")
    result = collection.find(
        {
            "required_age": 0,
            "user_score": {"$exists": True}
        },
        {"_id": 0, "name": 1, "user_score": 1}
    ).sort("user_score", -1).limit(5)
    for game in result:
        print(game)

consulta_5()

6 - Juegos mejor calificados en categorías de precio bajo, medio y alto (5 mejores calificados en cada categoría)

In [ ]:
def consulta_6():
    print("\nConsulta 6: Juegos mejor calificados en diferentes categorías de precio")

    # Precio bajo (hasta $10)
    low_price = collection.find(
        {"price": {"$lte": 10}},
        {"_id": 0, "name": 1, "price": 1, "user_score": 1}
    ).sort("user_score", -1).limit(5)
    print("\nBajo precio:")
    for game in low_price:
        print(game)

    # Precio medio (entre $11 y $30)
    mid_price = collection.find(
        {"price": {"$gt": 10, "$lte": 30}},
        {"_id": 0, "name": 1, "price": 1, "user_score": 1}
    ).sort("user_score", -1).limit(5)
    print("\nPrecio medio:")
    for game in mid_price:
        print(game)

    # Precio alto (más de $30)
    high_price = collection.find(
        {"price": {"$gt": 30}},
        {"_id": 0, "name": 1, "price": 1, "user_score": 1}
    ).sort("user_score", -1).limit(5)
    print("\nAlto precio:")
    for game in high_price:
        print(game)

consulta_6()

# Consistencia de datos



Inserción:

In [ ]:
new_game = {"name": "Juego de prueba", "price": 20, "user_score": 90, "required_age": 0}
collection.insert_one(new_game)

result = collection.find_one({"name": "Juego de prueba"})
print("Documento insertado:", result)

Actualización:

In [ ]:
collection.update_one({"name": "Juego de prueba"}, {"$set": {"price": 25}})

updated_game = collection.find_one({"name": "Juego de prueba"})
print("Documento actualizado:", updated_game)

Eliminación:

In [ ]:
collection.delete_one({"name": "Juego de prueba"})
deleted_game = collection.find_one({"name": "Juego de prueba"})
print("Documento eliminado:", deleted_game)  # Debe imprimir None si se eliminó correctamente

# Alta disponibilidad

Desde la consola de Docker, se debe ejecutar el siguiente código:

In [ ]:
docker stop mongo1

Ahora que mongo1 está desactivada, ejecutamos en la consola de MongoDB el siguiente comando:

In [ ]:
rs.status()

Acá podemos revisar que uno de los nodos secundarios pasa a ser primario.

Ahora probamos la consulta 2, para verificar que sigue funcionando normalmente.

In [ ]:
def consulta_2():
    print("\nConsulta 2: Juegos con precio entre $10 y $50 y al menos un DLC")
    result = collection.find(
        {
            "price": {"$gte": 10, "$lte": 50},
            "dlc_count": {"$gt": 0}
        },
        {"_id": 0, "name": 1, "dlc_count": 1}
    ).sort("dlc_count", -1).limit(10)

    for game in result:
        print(game)

consulta_2()

Una vez ejecutada la consulta, reiniciamos el nodo desactivado usando:

In [ ]:
docker start mongo1

Volvemos a revisar desde la consola de MongoDB el estado del replica-set:

In [ ]:
rs.status()